In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome
import time
import pickle
import tensorflow as tf
import re
from getpass import getpass

## Loading Model and tokenizer

In [2]:
model=tf.keras.models.load_model('lstmclassifier')
file= open('tokenizer', 'rb')
tokenizer = pickle.load(file)

## Implementing the cleaner

In [12]:
class ToxicCommentsRemover:
    login_status=0
    def __init__(self):
        #Starting web driver
        self.browser = webdriver.Chrome()
        self.browser.get("http://www.instagram.com")
    def login(self,username,password):
        #Login into the account
        username_field = self.browser.find_element_by_xpath("//input[@aria-label='Phone number, username, or email']")
        username_field.clear()
        username_field.send_keys(username)
        password_field = self.browser.find_element_by_xpath("//input[@aria-label='Password']")
        password_field.clear()
        password_field.send_keys(password)
        login=self.browser.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[4]/button')
        login.click()
        time.sleep(3)
        if self.browser.current_url=='https://www.instagram.com/':
            print("Wrong email/password")
        else:
            print("You are logged in")
            self.login_status=1
            
    
    def loadComments(self,link_to_the_post):
        self.browser.get(link_to_the_post)
        #LOADING EVERY COMMENT
        while 1:
            time.sleep(1)
            try:
                click=self.browser.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/div/div[3]/div[1]/ul/li/div/button')
                click.click()
            except:
                break
        #Getting Comments
        all_comments=[]
        comments=self.browser.find_elements_by_class_name('Mr508')
        for comment in comments:
            comment_text = comment.text
            comment_text = comment_text.split("\n")[1]
            #Preprocessing comment
            comment_text = self.preprocess_comment(comment_text)
            all_comments.append(comment_text)
        return all_comments    
    
    def clean_post(self,link_to_the_post):
        #Loading COmments
        if self.login_status==1:
            comments=self.loadComments(link_to_the_post)
            current=0
            #Iterating through comments
            for i in range(len(comments)):
                #Checking toxicity
                toxicity=self.check_toxicity(comments[i])
                if toxicity==1:
                    dot_path='//*[@id="react-root"]/section/main/div/div/article/div/div[3]/div[1]/ul/ul['+str(i+1-current)+']/div/li/div/div[2]/button'
                    userName = self.browser.find_element_by_xpath(dot_path)
                    self.browser.execute_script("arguments[0].click();", userName)
                    time.sleep(2)
                    try:
                        click=self.browser.find_element_by_xpath('/html/body/div[3]/div/div/div/div/button[2]')
                        click.click()
                    except:
                        click=self.browser.find_element_by_xpath('/html/body/div[4]/div/div/div/div/button[2]')
                        click.click()
                    time.sleep(1)
                    current+=1
                    self.browser.get(link_to_the_post)
            print('Your Post has been cleaned')     
    def preprocess_comment(self,comment):
        comment = re.sub(r"[!-()\"#/@;:<>{}=~|.?,]","",comment.lower())
        comment = re.sub("\\n"," ",comment)
        comment = re.sub("\d"," ",comment)
        comment=" ".join(comment.split())
        return [comment]
    def check_toxicity(self,comment):
        #Feeding comment into the lstm model
        comment = tokenizer.texts_to_sequences(comment)
        comment = tf.keras.preprocessing.sequence.pad_sequences(comment, maxlen=128)
        toxicity = model.predict(comment)
        if toxicity > 0.80:
            return 1
        else:
            return 0
        

In [10]:
username=input()
password=getpass()   #getpass to hide your password 


dankydankdonkey
········
You are logged in
Your Post has been cleaned


In [13]:
cleaner=ToxicCommentsRemover()
time.sleep(2)
cleaner.login(username,password)
time.sleep(1)
cleaner.clean_post("link_here")

You are logged in
Your Post has been cleaned
